In [ ]:
import pickle
import numpy as np
from flask import Flask, render_template, request

# Load the trained job recommendation model
model_path = "model.pkl"

try:
    with open(model_path, "rb") as model_file:
        model_data = pickle.load(model_file)
    print("Model loaded successfully:", model_data.keys())  # Debugging
except Exception as e:
    print("Error loading model:", e)
    model_data = None  # Prevents app from breaking

if model_data:
    vectorizer = model_data.get("vectorizer")
    similarity_matrix = model_data.get("similarity_matrix")
    job_titles = model_data.get("job_titles")
else:
    vectorizer = similarity_matrix = job_titles = None

# Initialize Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    if not model_data:
        return render_template('recommended.html', jobs=[], error="Model not loaded properly.")

    user_input = request.form['job'].strip()

    if not user_input:
        return render_template('recommended.html', jobs=[], error="Please enter a valid job title.")

    try:
        input_vector = vectorizer.transform([user_input])
        similarity_scores = np.dot(input_vector, similarity_matrix.T).toarray()[0]
        recommended_indices = similarity_scores.argsort()[::-1][:5]
        recommended_jobs = [job_titles[i] for i in recommended_indices]
    except Exception as e:
        return render_template('recommended.html', jobs=[], error=f"Error in recommendation: {e}")

    return render_template('recommended.html', jobs=recommended_jobs)

if __name__ == '__main__':
    app.run(debug=True)
